In [2]:
import os

import numpy as np
import pandas as pd

In [3]:
# 学習に時間がかかるのでCVはしない予定。
# train/valid/test

# test(valid)のanomalyを一部trainに

In [4]:
BASE_PATH = "/media/hiroki/working/datasets/DCASE2/dev_data/"
DATA_TYPE = os.listdir(BASE_PATH)

In [5]:
test_file_names = {}
test_sizes = {}
anomalies = {}
anomaly_sizes = {}
normal_sizes = {}

for d_type in DATA_TYPE:
    test_file_names[d_type] = os.listdir(BASE_PATH + d_type + "/test")
    test_sizes[d_type] = len(test_file_names[d_type])
    anomalies[d_type] = [file_name for file_name in test_file_names[d_type] if "anomaly" in file_name]
    anomaly_sizes[d_type] = len(anomalies[d_type])
    normal_sizes[d_type] = test_sizes[d_type] - anomaly_sizes[d_type]

In [6]:
normal_sizes

{'fan': 400,
 'pump': 400,
 'slider': 400,
 'ToyCar': 1400,
 'ToyConveyor': 2399,
 'valve': 400}

In [7]:
anomaly_sizes

{'fan': 1403,
 'pump': 432,
 'slider': 846,
 'ToyCar': 1007,
 'ToyConveyor': 1053,
 'valve': 455}

In [8]:
test_sizes

{'fan': 1803,
 'pump': 832,
 'slider': 1246,
 'ToyCar': 2407,
 'ToyConveyor': 3452,
 'valve': 855}

In [9]:
decision_anomaly_sizes = pd.Series(anomaly_sizes) * 0.05
decision_anomaly_sizes = decision_anomaly_sizes.round()

## 使うanomalyの数

In [10]:
# Size of all training anomaly data
decision_anomaly_sizes = pd.DataFrame(decision_anomaly_sizes).T

In [11]:
# Size of all training anomaly data
decision_anomaly_sizes

,fan,pump,slider,ToyCar,ToyConveyor,valve
0,70.0,22.0,42.0,50.0,53.0,23.0


In [12]:
four_id_columns = decision_anomaly_sizes.columns.drop("ToyConveyor")

In [13]:
# Size of training anomaly data per ID
decision_anomaly_sizes_per_id = decision_anomaly_sizes
decision_anomaly_sizes_per_id[four_id_columns] = decision_anomaly_sizes_per_id[four_id_columns] / 4
decision_anomaly_sizes_per_id["ToyConveyor"] = decision_anomaly_sizes_per_id["ToyConveyor"] / 3

In [14]:
# Size of training anomaly data per ID
decision_anomaly_sizes_per_id.round()

,fan,pump,slider,ToyCar,ToyConveyor,valve
0,18.0,6.0,10.0,12.0,18.0,6.0


# add_dataをtrain/validで分割

In [15]:
d_type

'valve'

7/3で分割

In [16]:
BASE_PATH = "/media/hiroki/working/datasets/DCASE2/add_dev_data/"
DATA_TYPE = os.listdir(BASE_PATH)

In [17]:
add_data_file_names = {}
add_data_sizes = {}

for d_type in DATA_TYPE:
    add_data_file_names[d_type] = os.listdir(BASE_PATH + d_type + "/train")
    add_data_sizes[d_type] = len(add_data_file_names[d_type])

In [18]:
add_data_sizes = pd.DataFrame(pd.Series(add_data_sizes)).T

In [19]:
# Size of add_data per Machine ID
add_data_sizes

,fan,pump,slider,ToyCar,ToyConveyor,valve
0,2846,2417,2370,3000,3000,2531


In [20]:
# Size of add_data valid
add_data_valid_sizes = (add_data_sizes * 0.3).round()

In [21]:
# Size of add_data valid
add_data_valid_sizes

,fan,pump,slider,ToyCar,ToyConveyor,valve
0,854.0,725.0,711.0,900.0,900.0,759.0


In [22]:
# Size of add_data per sound ID
add_data_valid_sizes_per_id = add_data_valid_sizes
add_data_valid_sizes_per_id[four_id_columns] = add_data_valid_sizes_per_id[four_id_columns] / 4
add_data_valid_sizes_per_id["ToyConveyor"] = add_data_valid_sizes_per_id["ToyConveyor"] / 3
add_data_valid_sizes_per_id = add_data_valid_sizes_per_id.astype('int')

In [23]:
add_data_valid_sizes_per_id

,fan,pump,slider,ToyCar,ToyConveyor,valve
0,213,181,177,225,300,189


In [24]:
toycar_machine_id = ["05","06","07"]
toyconveyor_machine_id = ["04""05","06"]
other_machine_id = ["01","03","05"]

valid_file_name = {}

for d_type in DATA_TYPE:
    files = add_data_file_names[d_type]
    valid_file_name[d_type] = {}
    
    if d_type == "ToyCar":
        for ID in toycar_machine_id:
            add_data_per_id = sorted([per_id for per_id in files if "id_"+ID in per_id])
            valid_size = add_data_valid_sizes_per_id[d_type].loc[0]
            valid_file_name[d_type][ID] = add_data_per_id[:valid_size]
    
    elif d_type == "ToyConveyor":
        for ID in toyconveyor_machine_id:
            add_data_per_id = sorted([per_id for per_id in files if "id_"+ID in per_id])
            valid_size = add_data_valid_sizes_per_id[d_type].loc[0]
            valid_file_name[d_type][ID] = add_data_per_id[:valid_size]
    
    else:
        for ID in other_machine_id:
            add_data_per_id = sorted([per_id for per_id in files if "id_"+ID in per_id])
            valid_size = add_data_valid_sizes_per_id[d_type].loc[0]
            valid_file_name[d_type][ID] = add_data_per_id[:valid_size]

In [25]:
valid_file_name

{'fan': {'01': ['normal_id_01_00000000.wav',
   'normal_id_01_00000001.wav',
   'normal_id_01_00000002.wav',
   'normal_id_01_00000003.wav',
   'normal_id_01_00000004.wav',
   'normal_id_01_00000005.wav',
   'normal_id_01_00000006.wav',
   'normal_id_01_00000007.wav',
   'normal_id_01_00000008.wav',
   'normal_id_01_00000009.wav',
   'normal_id_01_00000010.wav',
   'normal_id_01_00000011.wav',
   'normal_id_01_00000012.wav',
   'normal_id_01_00000013.wav',
   'normal_id_01_00000014.wav',
   'normal_id_01_00000015.wav',
   'normal_id_01_00000016.wav',
   'normal_id_01_00000017.wav',
   'normal_id_01_00000018.wav',
   'normal_id_01_00000019.wav',
   'normal_id_01_00000020.wav',
   'normal_id_01_00000021.wav',
   'normal_id_01_00000022.wav',
   'normal_id_01_00000023.wav',
   'normal_id_01_00000024.wav',
   'normal_id_01_00000025.wav',
   'normal_id_01_00000026.wav',
   'normal_id_01_00000027.wav',
   'normal_id_01_00000028.wav',
   'normal_id_01_00000029.wav',
   'normal_id_01_00000030.w